# Data Analysis

In this file, we proceed to develop and test the pipeline that handles our dataset.

We then proceed to analyse the output of said pipeline later on

### All the imports for all the things

In [52]:
import os
import sys
import pandas as pd
import datetime


import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import importlib


from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import callbacks  






import matplotlib.pyplot as plt
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go

# for translation of gene symbols
import mygene

import pathlib
import scipy

from sklearn.preprocessing import normalize, MinMaxScaler

# to import libs 
sys.path.append('../src/utils')
import  feature_selection, visualisation

sys.path.append('../src')
import data_handler



# datasets

# default path of the folder containing the salmon files
absolute_path = '/Users/aygalic/Thesis/data/quant/'  
metadata_path = '/Users/aygalic/Thesis/METADATA_200123.xlsx'  


mg = mygene.MyGeneInfo()

pd.options.display.width = 1000


### Some testing about gene name retrieval

In [3]:
# each couple of entries correspond to one patient, we are only interested in the "transcript" files
entries = os.listdir(absolute_path)
#entries_transcripts = [e for e in entries if "transcripts" in e ]
entries = [e for e in entries if "transcripts" in e ]
entry = entries[0]
entry

'PPMI-Phase2-IR2.41282.V02.0003241603.5104-SL-4884.longRNA-NEBKAP.salmon-gtf.transcripts.sf'

In [23]:
gene_names = data_handler.get_names(entry)
print(len(gene_names))
gene_names[0:3]

95309


['ENST00000335137.3|ENSG00000186092.4|OTTHUMG00000001094.1|OTTHUMT00000003223.1|OR4F5-001|OR4F5|918|CDS:1-918|',
 'ENST00000423372.3|ENSG00000237683.5|-|-|AL627309.1-201|AL627309.1|2661|UTR5:1-70|CDS:71-850|UTR3:851-2661|',
 'ENST00000426406.1|ENSG00000235249.1|OTTHUMG00000002860.1|OTTHUMT00000007999.1|OR4F29-001|OR4F29|995|UTR5:1-19|CDS:20-958|UTR3:959-995|']

In [51]:
name_df = pd.DataFrame([gn.split("|") for gn in gene_names])
name_df = name_df.set_axis([
    'trascript_id', 
    'gene_id', 
    'idk', 
    'idk', 
    'transcript_variant', 
    'symbol', 
    'length', 
    'untranslated_region_3', 
    'coding_region', 
    'untranslated_region_5', 
    'idk'], axis=1)

print(name_df[0:20])

         trascript_id            gene_id                   idk                   idk transcript_variant      symbol length untranslated_region_3   coding_region untranslated_region_5   idk
0   ENST00000335137.3  ENSG00000186092.4  OTTHUMG00000001094.1  OTTHUMT00000003223.1          OR4F5-001       OR4F5    918             CDS:1-918                                  None  None
1   ENST00000423372.3  ENSG00000237683.5                     -                     -     AL627309.1-201  AL627309.1   2661             UTR5:1-70      CDS:71-850         UTR3:851-2661      
2   ENST00000426406.1  ENSG00000235249.1  OTTHUMG00000002860.1  OTTHUMT00000007999.1         OR4F29-001      OR4F29    995             UTR5:1-19      CDS:20-958          UTR3:959-995      
3   ENST00000332831.2  ENSG00000185097.2  OTTHUMG00000002581.1  OTTHUMT00000007334.1         OR4F16-001      OR4F16    995             UTR5:1-19      CDS:20-958          UTR3:959-995      
4   ENST00000599533.1  ENSG00000269831.1               

In [50]:
print(name_df[['trascript_id', 'gene_id','transcript_variant', 'symbol']])

            trascript_id            gene_id transcript_variant      symbol
0      ENST00000335137.3  ENSG00000186092.4          OR4F5-001       OR4F5
1      ENST00000423372.3  ENSG00000237683.5     AL627309.1-201  AL627309.1
2      ENST00000426406.1  ENSG00000235249.1         OR4F29-001      OR4F29
3      ENST00000332831.2  ENSG00000185097.2         OR4F16-001      OR4F16
4      ENST00000599533.1  ENSG00000269831.1     AL669831.1-201  AL669831.1
...                  ...                ...                ...         ...
95304  ENST00000361335.1  ENSG00000212907.2        MT-ND4L-201     MT-ND4L
95305  ENST00000361381.2  ENSG00000198886.2         MT-ND4-201      MT-ND4
95306  ENST00000361567.2  ENSG00000198786.2         MT-ND5-201      MT-ND5
95307  ENST00000361681.2  ENSG00000198695.2         MT-ND6-201      MT-ND6
95308  ENST00000361789.2  ENSG00000198727.2         MT-CYB-201      MT-CYB

[95309 rows x 4 columns]


In [76]:
# we are interested in the gene codes without version

# getting of the "-xxx" at the end 
temp = [s.split('-')[:-1] for s in gene_names[4]]
versioned_symbols = ['-'.join(strings) for strings in temp]

# getting rid of the symbol
symbols = [sym.split(".")[0] for sym in versioned_symbols]
print(symbols[0:5])

['OR4F5', 'AL627309', 'OR4F29', 'OR4F16', 'AL669831']


In [19]:
# this makes us have an absord amount of duplicates
print(sum(pd.DataFrame(gene_names[0]).duplicated()))
#print(sum(pd.DataFrame(symbols).duplicated()))


0


In [82]:
dup_free = pd.DataFrame(symbols).drop_duplicates()
dup_free[0]

0           OR4F5
1        AL627309
2          OR4F29
3          OR4F16
4        AL669831
           ...   
95304     MT-ND4L
95305      MT-ND4
95306      MT-ND5
95307      MT-ND6
95308      MT-CYB
Name: 0, Length: 20575, dtype: object

In [83]:
qr = mg.querymany(dup_free[0], fields = ['genomic_pos', 'symbol'], scopes='symbol', species='human', verbose = True, as_dataframe = True)
qr

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

,_id,_score,symbol,genomic_pos.chr,genomic_pos.end,genomic_pos.ensemblgene,genomic_pos.start,genomic_pos.strand,notfound,genomic_pos
query,,,,,,,,,,
OR4F5,79501,25.651682,OR4F5,1,71585.0,ENSG00000186092,65419.0,1.0,NaN,NaN
AL627309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
OR4F29,729759,25.261646,OR4F29,1,451678.0,ENSG00000284733,450740.0,-1.0,NaN,NaN
OR4F16,81399,26.172808,OR4F16,1,686654.0,ENSG00000284662,685716.0,-1.0,NaN,NaN
AL669831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...
MT-ND4L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
MT-ND4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
MT-ND5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN


In [7]:
symbols = symbols.reset_index()
symbols = symbols.drop_duplicates(subset = ["query"])

In [9]:
sorted_sym = symbols.sort_values(['genomic_pos.chr', 'genomic_pos.start'], ascending=[True, True])

In [10]:
sorted_values = sorted_sym[['genomic_pos.chr', 'genomic_pos.start']].values
sorted_values

array([['1', 12010.0],
       ['1', 14404.0],
       ['1', 34554.0],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=object)

# Main function definition for the "genes" dataset

This is the main function that does the heavy lifting.

It has a lot of parametter to allow the user to process its dataset in many different ways.

The processing is four-fold : 

## Cell selection

Here we can decide to keep observations based on the provided meta-data.

## Feature engineering/sorting

Here we select genes based on criterions, we can also sort them based on their positions on the genome.

## Rescaling

Here we proceed to standardise or logscale the data

## Formating

Here the user can decide to output the dataset cellwise, or patient wise as time series.

### Just a bit of testing 

In [87]:
importlib.reload(feature_selection) # to allow modification of the script without restarting the whole session
importlib.reload(data_handler) # to allow modification of the script without restarting the whole session


as_time_series = False
dataset, index_names, n_genes, gene_names = data_handler.generate_dataset_transcripts(feature_selection_threshold = 2, 
                                                            batch_size = 32, 
                                                            retain_phases = "Both",
                                                            normalization = False,
                                                            as_time_series = as_time_series,
                                                            MT_removal = False)

NameError: name 'data_handler' is not defined


get the dataset as a numpy object for data manipulation

In [32]:
# get everything out of TensorFlow back to numpy/pandas
data = np.concatenate(list(dataset.as_numpy_iterator()), axis=0)
if(as_time_series):
    data = data.reshape(data.shape[0]*data.shape[1], -1)
    data = pd.DataFrame(data)
    data.columns = gene_names["name"]
    data.index = np.repeat(index_names, 5, axis=0)
else:
    data = pd.DataFrame(data)
    data.columns = gene_names["name"]
    data.index = index_names


In [33]:
print(data.head())

name      TOP1  ENSGR0000168939  ENSGR0000214717  ENSGR0000182484   
3442  0.314952         0.360238         0.136580         0.517728  \
3442  0.314498         0.420334         0.281522         0.490566   
3442  0.742467         0.354860         0.261963         0.379775   
3442  0.563241         0.467921         0.193135         0.520309   
3442  0.344854         0.334907         0.132422         0.420329   

name  ENSGR0000198223  ENSGR0000185960  ENSGR0000205755  ENSGR0000185291   
3442        -0.384073        -0.030902         0.462116         0.471284  \
3442        -0.136599         0.229172         0.495155         0.485320   
3442        -0.313197         0.386343         0.304953         0.417484   
3442        -0.159063        -0.018419         0.390361         0.396865   
3442        -0.456131        -0.360034         0.325078         0.271394   

name  ENSGR0000002586     KIF5A  ...    GARRE1    PTMAP2    TSEN34     MAPK4   
3442        -0.187010  0.360238  ...  0.179150  

In [39]:
importlib.reload(feature_selection) # to allow modification of the script without restarting the whole session

as_time_series = False

dataset, index_names, n_genes, gene_names = generate_dataset_transcripts(
    feature_selection_threshold = 2, 
    subsample= 100,
    retain_phases = "Both",
    normalization = False,
    as_time_series = as_time_series,
    MT_removal = False)

Retaining patients that are included in phases 1 & 2
retaining all patient who have at least passed the Base Line Visit...
loading samples...
loaded 27 samples
retriving symbols for genes
number of seq in the dataset : 27
keeping sample as is, no conversion to time series


In [62]:
pd.DataFrame([n.split("-")[0] for n in gene_names[4]]).duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
95304     True
95305     True
95306     True
95307     True
95308     True
Length: 95309, dtype: bool

In [89]:
# absurd amount of duplicates
print(sum(pd.DataFrame([s.rsplit('-', 1)[0] for s in gene_names[4]]).duplicated()))
print(sum(pd.DataFrame([s for s in gene_names[4]]).duplicated()))


index = 0
print(pd.DataFrame([s for s in gene_names[index]]))
print(sum(pd.DataFrame([s for s in gene_names[index]]).duplicated()))



74674
153


In [96]:
index = 0
print(pd.DataFrame([s for s in gene_names[index]]))
print(sum(pd.DataFrame([s for s in gene_names[index]]).duplicated()))


                       0
0      ENST00000335137.3
1      ENST00000423372.3
2      ENST00000426406.1
3      ENST00000332831.2
4      ENST00000599533.1
...                  ...
95304  ENST00000361335.1
95305  ENST00000361381.2
95306  ENST00000361567.2
95307  ENST00000361681.2
95308  ENST00000361789.2

[95309 rows x 1 columns]
0


In [60]:
[n.split(".")[0].split("-")[0] for n in gene_names[4]]

['OR4F5',
 'AL627309',
 'OR4F29',
 'OR4F16',
 'AL669831',
 'AL645608',
 'SAMD11',
 'SAMD11',
 'SAMD11',
 'SAMD11',
 'SAMD11',
 'AL645608',
 'NOC2L',
 'KLHL17',
 'KLHL17',
 'PLEKHN1',
 'PLEKHN1',
 'PLEKHN1',
 'PLEKHN1',
 'C1orf170',
 'C1orf170',
 'HES4',
 'HES4',
 'HES4',
 'ISG15',
 'AGRN',
 'AGRN',
 'RNF223',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'C1orf159',
 'TTLL10',
 'TTLL10',
 'TTLL10',
 'TTLL10',
 'TNFRSF18',
 'TNFRSF18',
 'TNFRSF18',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'SDF4',
 'SDF4',
 'SDF4',
 'SDF4',
 'B3GALT6',
 'FAM132A',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'UBE2J2',
 'SCNN1D',
 'SCNN1D',
 'SCNN1D',
 'SCNN1D',
 'SCNN1D',
 'SCNN1D',
 'SCNN1D',
 'ACAP3',
 'ACAP3',
 'ACAP3',

In [53]:
gene_names.iloc[0:20,4:]

,4,5,6,7,8,9,10
0,OR4F5-001,OR4F5,918,CDS:1-918,,None,None
1,AL627309.1-201,AL627309.1,2661,UTR5:1-70,CDS:71-850,UTR3:851-2661,
2,OR4F29-001,OR4F29,995,UTR5:1-19,CDS:20-958,UTR3:959-995,
3,OR4F16-001,OR4F16,995,UTR5:1-19,CDS:20-958,UTR3:959-995,
4,AL669831.1-201,AL669831.1,129,CDS:1-129,,None,None
5,AL645608.2-201,AL645608.2,57,CDS:1-57,,None,None
6,SAMD11-011,SAMD11,626,UTR5:1-89,CDS:90-626,,None
7,SAMD11-003,SAMD11,387,UTR5:1-60,CDS:61-387,,None
8,SAMD11-010,SAMD11,2551,UTR5:1-83,CDS:84-2129,UTR3:2130-2551,
9,SAMD11-001,SAMD11,2191,CDS:1-1769,UTR3:1770-2191,,None


In [41]:
print(gene_names)

                      0                  1                     2    
0      ENST00000335137.3  ENSG00000186092.4  OTTHUMG00000001094.1  \
1      ENST00000423372.3  ENSG00000237683.5                     -   
2      ENST00000426406.1  ENSG00000235249.1  OTTHUMG00000002860.1   
3      ENST00000332831.2  ENSG00000185097.2  OTTHUMG00000002581.1   
4      ENST00000599533.1  ENSG00000269831.1                     -   
...                  ...                ...                   ...   
95304  ENST00000361335.1  ENSG00000212907.2                     -   
95305  ENST00000361381.2  ENSG00000198886.2                     -   
95306  ENST00000361567.2  ENSG00000198786.2                     -   
95307  ENST00000361681.2  ENSG00000198695.2                     -   
95308  ENST00000361789.2  ENSG00000198727.2                     -   

                         3               4           5     6           7    
0      OTTHUMT00000003223.1       OR4F5-001       OR4F5   918   CDS:1-918  \
1                